In [1]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
import json
import numpy
from IPython.display import display
from subprocess import check_output
import sys
import traceback
from tqdm.notebook import tqdm, trange

In [2]:
techs = {
    "tech1": "1q_1o",
    "tech2": "k_per_terra",
    "tech3": "4pip",
    "tech4": "7vp",
    "tech5": "1o_1pw",
    "tech6": "1k_1c",
    "tech7": "3vp_per_gaia_place",
    "tech8": "4c",
    "tech9": "4pw",
}

adv_techs = {
    "advtech1": "3vp_per_fed_pass",
    "advtech2": "2vp_per_tech_bump",
    "advtech3": "1qic_5c_action",
    "advtech4": "2vp_per_mine",
    "advtech5": "3vp_per_rl_pass",
    "advtech6": "1o_per_sector",
    "advtech7": "1vp_per_terra_pass",
    "advtech8": "2vp_per_gaia",
    "advtech9": "4vp_per_ts",
    "advtech10": "2vp_per_sector",
    "advtech11": "3o_action",
    "advtech12": "5vp_per_fed",
    "advtech13": "3k_action",
    "advtech14": "3vp_per_mine_place",
    "advtech15": "3vp_per_ts_place",
}

feds = {
    "fed1": "12vp",
    "fed2": "qic",
    "fed3": "2pw",
    "fed4": "2o",
    "fed5": "6c",
    "fed6": "2k",
    "gleens": "gleens"
}

round_scorings = {
    "score1": "2vp_per_terra",
    "score2": "2vp_per_research_bump",
    "score3": "2vp_per_mine_place",
    "score4": "5vp_per_fed_place",
    "score5": "4vp_per_ts_place",
    "score6": "4vp_per_gaia_place",
    "score7": "5vp_per_3pip_place",
    "score8": "3vp_per_ts_place",
    "score9": "3vp_per_gaia_place",
    "score10": "5vp_per_3pip_place",
}

boosters = {

    "booster1": "1k_1o",
    "booster2": "2pwt_1o",
    "booster3": "1qic_2c",
    "booster4": "2c_terra",
    "booster5": "2pw_nav",
    "booster6": "1o_1vp_per_mine",
    "booster7": "1o_2vp_per_ts",
    "booster8": "1k_3vp_per_rl",
    "booster9": "4pw_4vp_per_3pip",
    "booster10": "4c_1vp_per_gaia",
}

action_names = {

    "power1": "3k_action",
    "power2": "2terra_action",
    "power3": "2o_action",
    "power4": "7c_action",
    "power5": "2k_action",
    "power6": "1terra_action",
    "power7": "2pt_action",
    "qic1": "4q_action",
    "qic2": "3q_action",
    "qic3": "2q_action"
}

bad_buildings = {"colony", 
                 "colonyShip", 
                 "tradeShip", 
                 "constructionShip", 
                 "researchShip",
                 "scout",
                 "frigate",
                 "battleShip",
                 "customsPost",
                 "tradePost"
                }

In [3]:
# for progressbar. Lines are very long jsons, so bash wc way better than native iteration
def wc(filename):
    return int(check_output(["wc", "-l", filename]).split()[0])

In [4]:
total_lines = wc('./data/game_data_raw.txt')
total_lines

35441

In [5]:
def parse_tree_moves(dat, tree, num_players):
    tech_slots = {
        'terra',
        'nav',
        'int',
        'gaia',
        'eco',
        'sci'
    }

    # keeping rounds and final scorings seperate for better analysis
    # starting to reach mem limits on server
    point_types = {
        'init',
        'bid',
        'charge',
        'booster',
        'round1',
        'round2',
        'round3',
        'round4',
        'round5',
        'round6',
        'fed',
        'adv',
        'tech_basic',
        'qic',
        'gaia',
        'gleens',
        'final1',
        'final2',
        'resource'
    }

    factions_in = []
    factions_rankings = []
    techs_in = {}
    faction_points = dict()  # dict of dicts

    # taken ranks to handle identical scores
    # player with lower id will have higher position (may have some impact on average positions)
    taken_rank = set()

    # get factions in
    for pos in range(num_players):
        faction = tree['players'][pos]['faction']
        factions_in.append(faction)
        points = dict()
        for point in point_types:
            points[point] = 0
        points['init'] = 10
        faction_points[faction] = points

        # get all scores
        scores = []
        for rank in range(num_players):
            scores.append(int(tree['players'][rank]['score']) + abs(int(tree['data']['players'][rank]['data']['bid'])))

        scores.sort()
        scores.reverse()

        # get position with bid factored
        rank_true = scores.index(
            int(tree['players'][pos]['score']) + abs(int(tree['data']['players'][pos]['data']['bid']))) + 1
        while rank_true in taken_rank:
            rank_true += 1
        taken_rank.add(rank_true)
        factions_rankings.append(rank_true)

    moves = tree['data']['advancedLog']

    for move in moves:
        if 'changes' in move.keys():
            changes = move['changes']
            faction = factions_in[move['player']]

            # charge
            if 'charge' in changes.keys() and 'vp' in changes['charge'].keys():
                faction_points[faction]['charge'] += changes['charge']['vp']

            # booster scoring
            for i in range(10):
                b = 'booster' + str(i + 1)
                if b in changes.keys() and 'vp' in changes[b].keys():
                    faction_points[faction]['booster'] += changes[b]['vp']
                    break

            # round
            for i in range(6):
                r = 'round' + str(i + 1)
                if r in changes.keys() and 'vp' in changes[r].keys():
                    faction_points[faction][r] += changes[r]['vp']
                    break

            # qic
            for i in range(3):
                q = 'qic' + str(i + 1)
                if q in changes.keys() and 'vp' in changes[q].keys():
                    faction_points[faction]['qic'] += changes[q]['vp']
                    break

            # feds
            if 'federation' in changes.keys() and 'vp' in changes['federation'].keys():
                faction_points[faction]['fed'] += changes['federation']['vp']

            # techs
            for slot in tech_slots:
                t = 'tech-' + slot
                if t in changes.keys() and 'vp' in changes[t].keys():
                    faction_points[faction]['tech_basic'] += changes[t]['vp']

                t = 'adv-' + slot
                if t in changes.keys() and 'vp' in changes[t].keys():
                    faction_points[faction]['adv'] += changes[t]['vp']

            for i in range(3):
                t = 'tech-free' + str(i + 1)
                if t in changes.keys() and 'vp' in changes[t].keys():
                    faction_points[faction]['tech_basic'] += changes[t]['vp']
                    break

                    # gaia
            # check that final scoring gaia (tech score) doesn't conflict with reaching top of track
            #     as they have the same key
            if 'gaia' in changes.keys() and 'vp' in changes['gaia'].keys() and 'move' in move.keys():
                faction_points[faction]['gaia'] += changes['gaia']['vp']

            # gleens
            if 'gleens' in changes.keys() and 'vp' in changes['gleens'].keys():
                faction_points[faction]['gleens'] += changes['gleens']['vp']

            # final scoring
            if 'final1' in changes.keys() and 'vp' in changes['final1'].keys():
                faction_points[faction]['final1'] += changes['final1']['vp']
            if 'final2' in changes.keys() and 'vp' in changes['final2'].keys():
                faction_points[faction]['final2'] += changes['final2']['vp']

            # spend
            if 'spend' in changes.keys() and 'vp' in changes['spend'].keys():
                faction_points[faction]['resource'] += changes['spend']['vp']

            # bid
            if 'bid' in changes.keys() and 'vp' in changes['bid'].keys():
                faction_points[faction]['bid'] += changes['bid']['vp']

    # tech score already in normal parsing

    for player in range(num_players):
        faction = factions_in[player]
        rank = factions_rankings[player]
        prefix = 'pos_' + str(rank) + '_score_'

        for pt in faction_points[faction].keys():
            dat[prefix + pt] = faction_points[faction][pt]

    return dat
        

In [6]:
def parse_tree_builds(dat, tree, pos, faction):
    moves = tree['data']['moveHistory']
    logs = tree['data']['advancedLog']

    cur_round = 0
    built = {
        'm': 0,
        'ts': 0,
        'lab': 0,
        'ac1': 0,
        'ac2': 0,
        'PI': 0,
        'gf': 0,
    }
    for log in logs:
        if 'round' in log.keys():
            cur_round = log['round']
            if cur_round > 1:
                for key in built.keys():

                    # catch all negative amounts
                    if built[key] < 0:
                        raise ValueError(
                            'negative structure amount found : ' + key + ' ' + faction + ' ' + str(cur_round))

                    dat[pos + 'buildings_r_' + str(cur_round - 1) + '_' + key] = built[key]

        if 'move' in log.keys():
            move = moves[log['move']]
            if 'build' in move and faction in move:
                struct = move.split('build')[1].split()[0]
                if struct in built.keys():
                    built[struct] += 1

                if struct == 'ts' and 'special' not in move:
                    built['m'] -= 1
                elif struct == 'ts' and 'special' in move:
                    built['lab'] -= 1
                elif struct == 'PI' and not faction == 'ivits':
                    if faction == 'bescods':
                        built['lab'] -= 1
                    else:
                        built['ts'] -= 1
                elif struct == 'lab':
                    built['ts'] -= 1
                elif struct == 'ac1' or struct == 'ac2':
                    if faction == 'bescods':
                        built['ts'] -= 1
                    else:
                        built['lab'] -= 1

    return dat

In [7]:
def parse_tree_techs(dat, tree, pos, faction):
    moves = tree['data']['moveHistory']
    logs = tree['data']['advancedLog']
    
    cur_round = 0
    tech_bumps = {
        'terra': 0,
        'nav': 0,
        'int': 0,
        'gaia': 0,
        'eco': 0,
        'sci': 0,
    }
    for log in logs:
        if 'round' in log.keys() or ('phase' in log.keys() and log['phase'] == 'endGame'):
            cur_round = 0
            if 'round' in log.keys():
                cur_round = log['round'] - 1
            if 'phase' in log.keys():
                cur_round = 6
            if cur_round > 0:
                for key in tech_bumps.keys():
                    dat[pos + 'tech_r_' + str(cur_round) + '_' + key] = tech_bumps[key]
        if 'move' in log.keys():
            move = moves[log['move']]
            if 'up' in move and faction in move:
                track = move.split('up')[1].split()[0]
                if track in tech_bumps.keys():
                    tech_bumps[track] += 1
    return dat

In [8]:
def parse_tree_actions(dat, tree, pos, faction):
    moves = tree['data']['moveHistory']
    logs = tree['data']['advancedLog']
    
    cur_round = 0
    actions = {
        'power1': 0,
        'power2': 0,
        'power3': 0,
        'power4': 0,
        'power5': 0,
        'power6': 0,
        'power7': 0,
        'qic1': 0,
        'qic2': 0,
        'qic3': 0
    }
    for log in logs:
        if 'round' in log.keys() or ('phase' in log.keys() and log['phase'] == 'endGame'):
            cur_round = 0
            if 'round' in log.keys():
                cur_round = log['round'] - 1
            if 'phase' in log.keys():
                cur_round = 6
            if cur_round > 0:
                for key in actions.keys():
                    dat[pos + 'action_r_' + str(cur_round) + '_' + action_names[key]] = actions[key]
                actions = dict.fromkeys(actions, 0)
        if 'move' in log.keys():
            move = moves[log['move']]
            if ' action' in move and faction in move:
                action = move.split(' action')[1].split()[0].replace('.','')
                if action in actions.keys():
                    actions[action] += 1
                    count = actions[action]
    return dat

In [9]:
"""
Parses non-expansion game data
"""
def parse_tree(tree, errors_set):
    if not tree['cancelled'] and tree['status'] == 'ended':
        try:
            dat = {}
            dat['id'] = tree['_id']

            # check for expansions
            if 'expansions' in tree['game'].keys() and len(tree['game']['expansions']) > 0:
                raise ValueError('this game is with expansions: ' + ''.join(tree['game']['expansions']))
            if 'expansions' in tree['data'].keys() and tree['data']['expansions'] != 0:
                raise ValueError('this game is with expansions: ' + str(tree['data']['expansions']))

            # some jsons dont have layout. must be before the site supported that feature
            # assume standard
            if 'options' in tree['data'].keys() and 'layout' in tree['data']['options'].keys():
                dat['map_layout'] = tree['data']['options']['layout']
            else:
                dat['map_layout'] = 'standard'

            # balanced factions?
            if 'options' in tree['game'].keys():
                if 'factionVariant' in tree['game']['options'].keys():
                    dat['balance_variant'] = tree['game']['options']['factionVariant']
                else:
                    dat['balance_variant'] = 'standard'

            num_players = tree['options']['setup']['nbPlayers']
            dat['num_players'] = num_players
            tot_elo = 0

            # boosters in game
            found_boosters = tree['data']['tiles']['boosters']
            for i in range(10):
                booster_name = 'booster' + str(i + 1)
                if booster_name in found_boosters.keys():
                    dat[boosters[booster_name]] = True
                else:
                    dat[boosters[booster_name]] = False

            # tech locations
            found_techs = tree['data']['tiles']['techs']
            for loc in found_techs.keys():
                name = found_techs[loc]['tile']
                if name in techs:
                    dat['tech_' + loc] = techs[name]
                elif name in adv_techs:
                    dat['tech_' + loc] = adv_techs[name]
                else:
                    raise NameError('unknown tech: ' + name)
                    # dat['tech_' + loc] = name

            # scorings
            found_scorings = tree['data']['tiles']['scorings']['round']
            for i in range(len(found_scorings)):
                dat['round_' + str(i + 1) + '_scoring'] = round_scorings[found_scorings[i]]
            dat['final_scoring_1'] = tree['data']['tiles']['scorings']['final'][0]
            dat['final_scoring_2'] = tree['data']['tiles']['scorings']['final'][1]

            # taken ranks to handle identical scores
            # player with lower id will have higher position (may have some impact on average positions)
            taken_rank = set()

            # player data
            for i in range(num_players):

                scores = []
                for rank in range(num_players):
                    scores.append(
                        int(tree['players'][rank]['score']) + abs(int(tree['data']['players'][rank]['data']['bid'])))

                scores.sort()
                scores.reverse()

                # position
                # pos = "pos_" + str(tree['players'][i]['ranking']) + "_"
                rank_true = scores.index(
                    int(tree['players'][i]['score']) + abs(int(tree['data']['players'][i]['data']['bid']))) + 1
                while rank_true in taken_rank:
                    rank_true += 1
                taken_rank.add(rank_true)
                pos = "pos_" + str(rank_true) + "_"

                # dropped or no
                dat[pos + 'dropped'] = tree['players'][i]['dropped']

                # elo
                elo = tree['players'][i]['elo']['initial']
                tot_elo += elo
                dat[pos + 'elo'] = elo

                # faction
                faction = tree['players'][i]['faction']
                dat[pos + 'faction'] = faction

                # score
                dat[pos + 'score'] = tree['players'][i]['score']

                # start pos
                dat[pos + 'start_pos'] = \
                    [i + 1 for i in range(len(tree['data']['setup'])) if tree['data']['setup'][i] == faction][0]

                # bid
                dat[pos + 'bid'] = tree['data']['players'][i]['data']['bid']

                # feds
                feds_taken = tree['data']['players'][i]['data']['tiles']['federations']
                dat[pos + 'feds_taken'] = len(feds_taken)
                for key in feds.keys():
                    dat[pos + 'fed_' + feds[key]] = 0
                for fed in feds_taken:
                    dat[pos + 'fed_' + feds[fed['tile']]] += 1

                # final buildings
                total_buildings = 0
                found_buildings = tree['data']['players'][i]['data']['buildings']
                for key in found_buildings.keys():
                    if key in bad_buildings:
                        if found_buildings[key] > 0:
                            raise ValueError('this is a game with expansions ' + key)
                        continue
                    elif key != 'gf' and key != 'sp':
                        dat[pos + 'build_' + key] = found_buildings[key]
                        total_buildings += found_buildings[key]
                dat[pos + 'num_structures'] = total_buildings

                # research
                tech_score = 0
                found_research = tree['data']['players'][i]['data']['research']
                for key in found_research.keys():
                    if key == 'dip':
                        if found_research[key] > 0:
                            # normal games seem to have this key now also
                            raise ValueError('this is a game with expansions ' + key)
                    else:
                        dat[pos + 'research_level_' + key] = found_research[key]
                    if found_research[key] > 2:
                        tech_score += (4 * (found_research[key] - 2))
                dat[pos + 'tech_score'] = tech_score

                # techs taken
                total_techs = 0
                found_techs = tree['data']['players'][i]['data']['tiles']['techs']
                for key in techs.keys():
                    dat[pos + 'tech_taken_' + techs[key]] = False
                for key in adv_techs.keys():
                    dat[pos + 'adv_tech_taken_' + adv_techs[key]] = False
                for tech in found_techs:
                    total_techs += 1
                    name = tech['tile']
                    if name in techs.keys():
                        dat[pos + 'tech_taken_' + techs[name]] = True
                    elif name in adv_techs.keys():
                        dat[pos + 'adv_tech_taken_' + adv_techs[name]] = True
                    else:
                        dat[pos + 'tech_taken_' + adv_techs[name]] = True
                dat[pos + 'total_techs_taken'] = total_techs

                # buildings
                dat = parse_tree_builds(dat, tree, pos, faction)
                
                # r1 techs
                dat = parse_tree_techs(dat, tree, pos, faction)
                
                # pwr/qic actions
                dat = parse_tree_actions(dat, tree, pos, faction)

            dat = parse_tree_moves(dat, tree, num_players)
            dat['average_elo'] = tot_elo / num_players
            dat = pd.DataFrame(dat, index=[0])
            return True, dat, num_players, errors_set
        except:
            errors_set[tree['_id']] = traceback.format_exc()
            return False, "", "", errors_set

    else:
        return False, "", "", errors_set


In [10]:
'''
ToDo: parallelize this so its faster
'''

test_size = -1

with open("./data/game_data_raw.txt", "r") as game_data_raw:
    
    pbar_total = total_lines
    if (test_size > 0):
        pbar_total = test_size
    pbar = tqdm(total=pbar_total)
    
    two_players = None
    three_players = None
    four_players = None
    
    errors_set = dict()
    
    lines = game_data_raw.readlines()
    line_num = 0
    for idx, line in enumerate(lines):
        game_tree = json.loads(line)
        success, df, num_player, errors_set = parse_tree(game_tree, errors_set)
        if success:
            if num_player == 2:
                if two_players is not None:
                    two_players = pd.concat([two_players, df], axis=0, join="outer", ignore_index=True)
                else:
                    two_players = df
            elif num_player == 3:
                if three_players is not None:
                    three_players = pd.concat([three_players, df], axis=0, join="outer", ignore_index=True)
                else:
                    three_players = df
            else:
                if four_players is not None:
                    four_players = pd.concat([four_players, df], axis=0, join="outer", ignore_index=True)
                else:
                    four_players = df
        pbar.update(1)
        if (test_size > 0 and idx >= test_size-1): break

  0%|          | 0/35441 [00:00<?, ?it/s]

In [11]:
len(errors_set)

9669

In [12]:
# monitor parsing errors
err_types = set(errors_set.values())
err_to_count = {}
for err in err_types:
    count = sum(map(str(err).__eq__, errors_set.values()))
    err_to_count[err] = count

for key in err_to_count.keys():
    print(key)
    print(err_to_count[key])
    print('\n------')

Traceback (most recent call last):
  File "/var/folders/mj/1t1dl94s629468zg9hyjx8b00000gp/T/ipykernel_27313/1229003372.py", line 72, in parse_tree
    int(tree['players'][rank]['score']) + abs(int(tree['data']['players'][rank]['data']['bid'])))
KeyError: 'bid'

44

------
Traceback (most recent call last):
  File "/var/folders/mj/1t1dl94s629468zg9hyjx8b00000gp/T/ipykernel_27313/1229003372.py", line 52, in parse_tree
    raise NameError('unknown tech: ' + name)
NameError: unknown tech: tech-ship0

9004

------
Traceback (most recent call last):
  File "/var/folders/mj/1t1dl94s629468zg9hyjx8b00000gp/T/ipykernel_27313/1229003372.py", line 90, in parse_tree
    elo = tree['players'][i]['elo']['initial']
KeyError: 'elo'

42

------
Traceback (most recent call last):
  File "/var/folders/mj/1t1dl94s629468zg9hyjx8b00000gp/T/ipykernel_27313/1229003372.py", line 12, in parse_tree
    raise ValueError('this game is with expansions: ' + ''.join(tree['game']['expansions']))
ValueError: this game i

In [13]:
# get examples to check out jsons
# JSON: https://www.boardgamers.space/api/gameplay/ + key
# see if game loads (usually doesnt if error): https://www.boardgamers.space/game/ + key
for err in err_types:
    for key in errors_set.keys():
        if errors_set[key] == err:
            print('Game Name: ' + str(key) + ' Total Error Count: ' + str(err_to_count[err]) + '\n')
            print(err)
            print('\n------')
            break

Game Name: Gaiapolis Total Error Count: 44

Traceback (most recent call last):
  File "/var/folders/mj/1t1dl94s629468zg9hyjx8b00000gp/T/ipykernel_27313/1229003372.py", line 72, in parse_tree
    int(tree['players'][rank]['score']) + abs(int(tree['data']['players'][rank]['data']['bid'])))
KeyError: 'bid'


------
Game Name: big-mine-3378 Total Error Count: 9004

Traceback (most recent call last):
  File "/var/folders/mj/1t1dl94s629468zg9hyjx8b00000gp/T/ipykernel_27313/1229003372.py", line 52, in parse_tree
    raise NameError('unknown tech: ' + name)
NameError: unknown tech: tech-ship0


------
Game Name: Blue-desire-2313 Total Error Count: 42

Traceback (most recent call last):
  File "/var/folders/mj/1t1dl94s629468zg9hyjx8b00000gp/T/ipykernel_27313/1229003372.py", line 90, in parse_tree
    elo = tree['players'][i]['elo']['initial']
KeyError: 'elo'


------
Game Name: Tempting-reaction-434 Total Error Count: 174

Traceback (most recent call last):
  File "/var/folders/mj/1t1dl94s62946

In [14]:
two_players = two_players.drop_duplicates(subset=['id'])
three_players = three_players.drop_duplicates(subset=['id'])
four_players = four_players.drop_duplicates(subset=['id'])

In [15]:
len(two_players) + len(three_players) + len(four_players)

19719

In [16]:
two_players.to_pickle("data/two_players_data")
three_players.to_pickle("data/three_players_data")
four_players.to_pickle("data/four_players_data")

In [17]:
two_players.to_csv("data/two_players_data.csv", index=False)
three_players.to_csv("data/three_players_data.csv", index=False)
four_players.to_csv("data/four_players_data.csv", index=False)

In [18]:
for col in two_players.columns:
    print(col)

id
map_layout
balance_variant
num_players
1k_1o
2pwt_1o
1qic_2c
2c_terra
2pw_nav
1o_1vp_per_mine
1o_2vp_per_ts
1k_3vp_per_rl
4pw_4vp_per_3pip
4c_1vp_per_gaia
tech_terra
tech_nav
tech_int
tech_gaia
tech_eco
tech_sci
tech_free1
tech_free2
tech_free3
tech_adv-terra
tech_adv-nav
tech_adv-int
tech_adv-gaia
tech_adv-eco
tech_adv-sci
round_1_scoring
round_2_scoring
round_3_scoring
round_4_scoring
round_5_scoring
round_6_scoring
final_scoring_1
final_scoring_2
pos_1_dropped
pos_1_elo
pos_1_faction
pos_1_score
pos_1_start_pos
pos_1_bid
pos_1_feds_taken
pos_1_fed_12vp
pos_1_fed_qic
pos_1_fed_2pw
pos_1_fed_2o
pos_1_fed_6c
pos_1_fed_2k
pos_1_fed_gleens
pos_1_build_m
pos_1_build_ts
pos_1_build_lab
pos_1_build_PI
pos_1_build_ac1
pos_1_build_ac2
pos_1_num_structures
pos_1_research_level_terra
pos_1_research_level_nav
pos_1_research_level_int
pos_1_research_level_gaia
pos_1_research_level_eco
pos_1_research_level_sci
pos_1_tech_score
pos_1_tech_taken_1q_1o
pos_1_tech_taken_k_per_terra
pos_1_tech_taken

In [19]:
four_players['balance_variant'].value_counts()

standard         4961
more-balanced    1610
beta              143
Name: balance_variant, dtype: int64